In [ ]:
import os
import sys
from bs4 import BeautifulSoup
import pymysql
import requests
import re
import time

from Utils.bulk_insert import BulkInsert

In [ ]:
db_params = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'daigo1123',
    'database': 'dev_netkeiba',
    'port': 3306,
    'charset': 'utf8'
}
con = pymysql.connect(**db_params)

parameters = {

    # parameters about scraping
    'URL_ABOUT_NETKEIBA': {
        'RACE_TABLE': 'https://race.netkeiba.com/?pid=race_old&id=c',
        'RACE_RESULT': 'https://race.netkeiba.com/?pid=race&id=c{RACE_ID}&mode=result',
        'RACE_PAST5_RESULT': 'https://race.netkeiba.com/?pid=race&id=c{RACE_ID}&mode=shutuba'
    },

    # parameters about model training

    # col names in database tables
    'TABLE_COL_NAMES': {
        'race_master': [
            'race_id',
            'race_title',
            'race_coure',
            'race_weather',
            'race_condition',
            'race_year',
            'race_month',
            'race_date',
            'race_dow',
            'starting_time',
            'race_info_1',
            'race_info_2',
            'race_info_3'
        ],
        'race_table_info': [
            'race_id',
            'bracket_num',
            'horse_num',
            'horse_name',
            'horse_age',
            'horse_sex',
            'weight_penalty',
            'jockey_name',
            'href_to_jockey',
            'owner_name',
            'href_to_owner',
            'horse_weight',
            'horse_weight_increment',
            'win_odds',
            'popularity_order'
        ],
        'race_result_info': [
            'race_id',
            'bracket_num',
            'horse_num',
            'arrival_time',
            'arrival_diff',
            'arrrival_order'
        ],
        'race_refund_info': [
            'race_id',
            'refund_type',
            'groupby_index',
            'bracket_num',
            'refund_yen',
            'popularity_order'
        ],
        'race_past_5_result_info':[
            'race_id',
            'bracket_num',
            'horse_num',
            'past_1_order',
            'past_2_order',
            'past_3_order',
            'past_4_order',
            'past_5_order'
        ]
    },

    # col names in dataframe
    'DATAFRAME_COL_NAMES': {

    }
}

## Get info about race prior table 

In [ ]:
def _get_num_str(num):
    num_str = str(num) if num >= 10 else '0' + str(num)
    return num_str

In [ ]:
def make_race_id_and_target_url(event_year, event_place, event_month, event_time, event_race):
    race_id = str(event_year) + _get_num_str(event_place) + _get_num_str(event_month) + _get_num_str(event_time) + _get_num_str(event_race)
    target_url = parameters['URL_ABOUT_NETKEIBA']['RACE_TABLE'] + race_id
    
    return race_id, target_url

In [ ]:
def _fetchall_and_make_list_by(query):
    try:
        cursor = con.cursor()
        cursor.execute(query)
        fetch_result = cursor.fetchall()
        fetch_result_list = [item for item in fetch_result]
        cursor.close()
        return fetch_result_list
    except Exception as e:
        print(e)

def _is_the_race_id_existing_in_master(race_id):
    query = """
        SELECT race_id 
        FROM race_master
        WHERE race_id = "{RACE_ID}";
    """.format(RACE_ID=race_id)
    race_id_list_in_master = _fetchall_and_make_list_by(query)
    if race_id_list_in_master is None:
        return True
    else:
        return False

In [ ]:
def _extract_common_info(soup, race_id):
    race_title = soup.find('div', class_='data_intro').find('h1').text.replace(u'\xa0',u' ')
    
    race_coure = soup.find('div', class_='data_intro').find_all('p')[0].text.replace(u'\xa0',u' ')
    
    race_weather = soup.find('div', class_='data_intro').find_all('p')[1].text.replace(u'\xa0',u' ').split('/')[0]
    race_weather = re.search('天気：(.*)', race_weather).group(1)
    
    race_condition = soup.find('div', class_='data_intro').find_all('p')[1].text.replace(u'\xa0',u' ').split('/')[1]
    race_condition = re.search('馬場：(.*)', race_condition).group(1)
    
    starting_time = soup.find('div', class_='data_intro').find_all('p')[1].text.replace(u'\xa0',u' ').split('/')[2]
    starting_time = re.search('発走：(.*)', starting_time).group(1)
    
    race_date_info = soup.find('div', class_='data_intro').find('div', class_='race_otherdata').find_all('p')[0].text
    race_year = re.split('/|\(|\)', race_date_info)[0]
    race_month = re.split('/|\(|\)', race_date_info)[1]
    race_date = re.split('/|\(|\)', race_date_info)[2]
    race_dow = re.split('/|\(|\)', race_date_info)[3]
    
    race_info_1 = soup.find('div', class_='data_intro').find('div', class_='race_otherdata').find_all('p')[1].text.replace(u'\xa0',u' ')
    race_info_2 = soup.find('div', class_='data_intro').find('div', class_='race_otherdata').find_all('p')[2].text.replace(u'\xa0',u' ')
    race_info_3 = soup.find('div', class_='data_intro').find('div', class_='race_otherdata').find_all('p')[3].text.replace(u'\xa0',u' ')
    
    return race_id, race_title, race_coure, race_weather, race_condition, race_year, race_month, race_date, race_dow, starting_time, race_info_1, race_info_2, race_info_3

In [ ]:
def _extract_race_table(soup, race_id):
    this_race_table_info = []
    
    table_length = len(soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')) 
    for row in range(3, table_length):
        bracket_num = int(soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[0].text)
        horse_num = int(soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[1].text)
        horse_name = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[3].find('a').text
        sex_and_age = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[4].text
        horse_age = int(re.sub("\\D", "", sex_and_age))
        horse_sex = re.match('[0-9a-zA-Zあ-んア-ン一-鿐]', sex_and_age).group()
        weight_penalty = float(soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[5].text)
        jockey_name = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[6].text
        href_to_jockey = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[6].find('a').attrs['href']
        owner_name = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[7].text
        href_to_owner = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[7].find('a').attrs['href']
        horse_weight_info = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[8].text
        if horse_weight_info != '':
            horse_weight = int(re.split('\(|\)', horse_weight_info)[0])
            horse_weight_increment = re.split('\(|\)', horse_weight_info)[1]
        else:
            horse_weight = ''
            horse_weight_increment = ''

        win_odds = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[9].text
        popularity_order = soup.find('table', class_='race_table_old nk_tb_common').find_all('tr')[row].find_all('td')[10].text
        
        this_race_table_info.append([
            race_id,
            bracket_num,
            horse_num,
            horse_name,
            horse_age,
            horse_sex,
            weight_penalty,
            jockey_name,
            href_to_jockey,
            owner_name,
            href_to_owner,
            horse_weight,
            horse_weight_increment,
            win_odds,
            popularity_order
        ])
        
    return this_race_table_info

In [ ]:
def _execute_query(query):
    try:
        cursor = con.cursor()
        cursor.execute(query)
        cursor.close()
        con.commit()
    except Exception as e:
        print(e)

# def _truncate_target_rows(race_id):
#     queries = [
#         'TRUNCATE TABLE race_master WHERE race_id = "{RACE_ID}";'.format(RACE_ID=race_id),
#         'TRUNCATE TABLE race_table_info WHERE race_id = "{RACE_ID}";'.format(RACE_ID=race_id)
#     ]
#     for query in queries:
#         print(query)
#         _execute_query(query)

def _bulk_insert(insert_list, target_table_name, insert_col_names):
    try:
        bi = BulkInsert(con)
        bi.execute(insert_data=insert_list, target_table=target_table_name, col_names=insert_col_names)
    except TypeError as e:
        print(e)
        raise TypeError

In [ ]:
def get_and_insert_race_master_and_table_info():
    for event_year in range(2019, 2020):
        for event_place in range(1,2):
            for event_month in range(1, 2):
                for event_time in range(1, 2):
                    for event_race in range(1, 2):
                        race_master_list = []
                        race_table_info_list = []
                        race_id, target_url = make_race_id_and_target_url(event_year, event_place, event_month, event_time, event_race)                    
                        html = requests.get(target_url)
                        html.encoding = 'EUC-JP'
                        soup = BeautifulSoup(html.text, 'html.parser')

                        if not soup.find_all('table', attrs={'class', 'race_table_old nk_tb_common'}):
                            print('Target URL to requests ', target_url, 'does not exist.')
                            continue

                        print('Target URL to requests: ', target_url)
                        try:
                            race_master_list.append(_extract_common_info(soup, race_id))
                        except AttributeError:
                            print('\t This URL has no master info')
                        race_table_info_list = race_table_info_list + _extract_race_table(soup, race_id)
                        _bulk_insert(race_master_list, 'race_master', parameters['TABLE_COL_NAMES']['race_master'])
                        _bulk_insert(race_table_info_list, 'race_table_info', parameters['TABLE_COL_NAMES']['race_table_info'])                        
                        
                        time.sleep(1)
                
#     return race_master_list, race_table_info_list

In [ ]:
# race_master_list, race_table_info_list = get_and_insert_race_master_and_table_info()
get_and_insert_race_master_and_table_info()

In [ ]:
# print(pd.DataFrame(race_master_list).shape)
# print(pd.DataFrame(race_table_info_list).shape)

In [ ]:
# pd.DataFrame(race_master_list).to_csv('race_master.csv', index=False)

In [ ]:
# pd.DataFrame(race_table_info_list).to_csv('race_table_info_list.csv', index=False)

In [ ]:
event_year = 2019
event_place = 8
event_month =5
event_time = 4
event_race = 11

race_id, target_url = make_race_id_and_target_url(event_year, event_place, event_month, event_time, event_race)
print(race_id)
print('Target URL to requests: ', target_url)

html = requests.get(target_url)
html.encoding = 'EUC-JP'
soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
try:
    race_master_list = _extract_common_info(soup, race_id)
    race_table_info_list = _extract_race_table(soup, race_id)
except (AttributeError, ValueError):
    print('\t This URL has no master info')

In [ ]:
# _bulk_insert(race_id, race_master_list, 'race_master', parameters['TABLE_COL_NAMES']['race_master'])

In [ ]:
# _bulk_insert(race_id, race_table_info_list, 'race_table_info', parameters['TABLE_COL_NAMES']['race_table_info'])                        

## Get info about race result

In [ ]:
def _fetchall_and_make_list_by(query, con):
    try:
        cursor = con.cursor()
        cursor.execute(query)
        fetch_result = cursor.fetchall()
        fetch_result_list = [item for item in fetch_result]
        cursor.close()
        return fetch_result_list
    except Exception as e:
        print(e)

In [ ]:
def _make_target_url_about_race_result(race_id):
        return parameters['URL_ABOUT_NETKEIBA']['RACE_RESULT'].format(RACE_ID=race_id)

In [ ]:
def _extract_race_ids_in_master():
    query = """
        SELECT DISTINCT race_id 
        FROM race_master
        WHERE race_id NOT IN (SELECT DISTINCT race_id FROM race_result_info);
    """
    result = _fetchall_and_make_list_by(query, con)
    return result

In [ ]:
def _extract_race_result_info(soup, race_id):
    this_race_result_info = []
    table_length = len(soup.find('table', class_='race_table_01 nk_tb_common').find_all('tr'))
    
    for row in range(1, table_length):
        arrrival_order  = soup.find('table', class_='race_table_01 nk_tb_common').find_all('tr')[row].find_all('td')[0].text
        bracket_num = soup.find('table', class_='race_table_01 nk_tb_common').find_all('tr')[row].find_all('td')[1].text
        horse_num = soup.find('table', class_='race_table_01 nk_tb_common').find_all('tr')[row].find_all('td')[2].text
        arrival_time = soup.find('table', class_='race_table_01 nk_tb_common').find_all('tr')[row].find_all('td')[7].text
        arrival_diff = soup.find('table', class_='race_table_01 nk_tb_common').find_all('tr')[row].find_all('td')[8].text
        
        this_race_result_info.append([
            race_id,
            bracket_num,
            horse_num,
            arrival_time,
            arrival_diff,
            arrrival_order
        ])

    return this_race_result_info

In [ ]:
def _extract_race_refund_info(soup, race_id):
    empty_refund_list = []
    refund_table_list = soup.find('dd', class_='fc').find_all('tr')
    for i in range(len(refund_table_list)):
        refund_table = refund_table_list[i]
        refund_type = refund_table.find('th').text

        if refund_type == '単勝':
            empty_refund_list.append(
                [race_id, '単勝', 1,  
                 int(refund_table.find_all('td')[0].text),
                 int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                 int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )

        elif refund_type == '複勝':
            empty_refund_list.append(
                [race_id, '複勝', 1,  
                 int(refund_table.find_all('td')[0].text[:2]),
                 int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                 int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, '複勝', 2,  
                 int(refund_table.find_all('td')[0].text[2:4]),
                 int(refund_table.find_all('td')[1].text.split('円')[1].replace(',', '')),
                 int(refund_table.find_all('td')[2].text.split('人気')[1].replace(',', ''))]
            )
            try:
                empty_refund_list.append(
                    [race_id, '複勝', 3, 
                     int(refund_table.find_all('td')[0].text[4:6]),
                     int(refund_table.find_all('td')[1].text.split('円')[2].replace(',', '')),
                     int(refund_table.find_all('td')[2].text.split('人気')[2].replace(',', ''))]
                )
            except ValueError:
                pass

        elif refund_type == '枠連':
            empty_refund_list.append(
                [race_id, '枠連', 1,  
                 int(refund_table.find_all('td')[0].text.split('-')[0]),
                 int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                 int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))],
            )
            empty_refund_list.append(
                [race_id, '枠連', 1,  
                 int(refund_table.find_all('td')[0].text.split('-')[1]),
                 int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                 int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )

        elif refund_type == '馬連':
            empty_refund_list.append(
                [race_id, '馬連', 1,  
                int(refund_table.find_all('td')[0].text.split('-')[0]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))],
            )
            empty_refund_list.append(
                [race_id, '馬連', 1,  
                int(refund_table.find_all('td')[0].text.split('-')[1]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )   
            
        elif refund_type == 'ワイド':
            empty_refund_list.append(
                [race_id, 'ワイド', 1,  
                int(refund_table.find_all('td')[0].text[:2]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, 'ワイド', 1,  
                int(refund_table.find_all('td')[0].text[5:7]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, 'ワイド', 2,  
                int(refund_table.find_all('td')[0].text[7:9]),
                int(refund_table.find_all('td')[1].text.split('円')[1].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[1].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, 'ワイド', 2,  
                int(refund_table.find_all('td')[0].text[12:14]),
                int(refund_table.find_all('td')[1].text.split('円')[1].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[1])],            
            )
            empty_refund_list.append(
                [race_id, 'ワイド', 3,  
                int(refund_table.find_all('td')[0].text[14:16]),
                int(refund_table.find_all('td')[1].text.split('円')[2].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[2].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, 'ワイド', 3,  
                int(refund_table.find_all('td')[0].text[19:22]),
                int(refund_table.find_all('td')[1].text.split('円')[2].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[2].replace(',', ''))]
            )   
            
        elif refund_type == '馬単':
            empty_refund_list.append(
                [race_id, '馬単', 1,
                int(refund_table.find_all('td')[0].text.split('→')[0]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, '馬単', 1,
                int(refund_table.find_all('td')[0].text.split('→')[1]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0])]             
            )
            
        elif refund_type == '三連複':
            empty_refund_list.append(
                [race_id, '三連複', 1,
                int(refund_table.find_all('td')[0].text.split('-')[0]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, '三連複', 1,
                int(refund_table.find_all('td')[0].text.split('-')[1]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )        
            empty_refund_list.append(
                [race_id, '三連複', 1,
                int(refund_table.find_all('td')[0].text.split('-')[2]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))] 
            )       
            
        elif refund_type == '三連単':
            empty_refund_list.append(
                [race_id, '三連単', 1,
                int(refund_table.find_all('td')[0].text.split('→')[0]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )
            empty_refund_list.append(
                [race_id, '三連単', 1,
                int(refund_table.find_all('td')[0].text.split('→')[1]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )        
            empty_refund_list.append(
                [race_id, '三連単', 1,
                int(refund_table.find_all('td')[0].text.split('→')[2]),
                int(refund_table.find_all('td')[1].text.split('円')[0].replace(',', '')),
                int(refund_table.find_all('td')[2].text.split('人気')[0].replace(',', ''))]
            )                

    return empty_refund_list

In [ ]:
def get_race_result_info():
    existing_race_ids_in_master = _extract_race_ids_in_master()
    
    for id_idx in range(len(existing_race_ids_in_master)):
        race_id = existing_race_ids_in_master[id_idx][0]
        target_url = _make_target_url_about_race_result(race_id)
        
        html = requests.get(target_url)
        html.encoding = 'EUC-JP'
        soup = BeautifulSoup(html.text, 'html.parser')

        if not soup.find_all('table', attrs={'class', 'race_table_01 nk_tb_common'}):
            print('Target URL to requests ', target_url, 'does not exist.')
            break

        print('Target URL to requests: ', target_url)
        race_result_info_list = _extract_race_result_info(soup, race_id)
        race_refund_info_list = _extract_race_refund_info(soup, race_id)
        
        _bulk_insert(race_result_info_list, 'race_result_info', parameters['TABLE_COL_NAMES']['race_result_info'])
        _bulk_insert(race_refund_info_list, 'race_refund_info', parameters['TABLE_COL_NAMES']['race_refund_info'])                        

        time.sleep(1)

In [ ]:
get_race_result_info()

## Get info about past 5 race result

In [ ]:
def _make_target_url_about_past_5_race_result(race_id):
    return parameters['URL_ABOUT_NETKEIBA']['RACE_PAST5_RESULT'].format(RACE_ID=race_id)

In [ ]:
def _extract_past_5_race_result(soup, race_id):
    this_race_past5_result_info = []
    table_element = soup.find('table', class_='race_table_01 nk_tb_common shutuba_table').find_all('tr')
    table_length = len(table_element)

    for row in range(1, table_length):
        bracket_num = table_element[row].find_all('td')[0].text
        horse_num = table_element[row].find_all('td')[1].text

        order_list = []
        for col in range(6, 11):
            try:
                order = table_element[row].find_all('td')[col].find('span', class_='order').text
            except AttributeError:
                order = ''
            order_list += [order]

        this_race_past5_result_info.append([race_id, bracket_num, horse_num] + order_list)
    return this_race_past5_result_info

In [ ]:
def get_past_5_race_result_info():
    existing_race_ids_in_master = _extract_race_ids_in_master()
    
    for id_idx in range(len(existing_race_ids_in_master)):
        race_id = existing_race_ids_in_master[id_idx][0]
        target_url = _make_target_url_about_race_result(race_id)
        
        html = requests.get(target_url)
        html.encoding = 'EUC-JP'
        soup = BeautifulSoup(html.text, 'html.parser')

        if not soup.find_all('table', attrs={'class', 'race_table_01 nk_tb_common shutuba_table'}):
            print('Target URL to requests ', target_url, 'does not exist.')
            break

        print('Target URL to requests: ', target_url)
        race_past5_result_info_list = _extract_past_5_race_result(soup, race_id)        
        _bulk_insert(race_past5_result_info_list, 'race_past_5_result_info', 
                     parameters['TABLE_COL_NAMES']['race_past_5_result_info'])                        

        time.sleep(1)

In [ ]:
existing_race_ids_in_master = _extract_race_ids_in_master()
existing_race_ids_in_master[0]

In [ ]:
id_idx = 0
race_id = existing_race_ids_in_master[id_idx][0]
race_id

In [ ]:
target_url = _make_target_url_about_past_5_race_result(race_id)
target_url

In [ ]:
html = requests.get(target_url)
html.encoding = 'EUC-JP'
soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
_extract_past_5_race_result(soup, race_id)